author: leezeeyee   
date: 2021/3/17   
link: [github](https://github.com/easilylazy/pattern-recognition) 

In [1]:
import numpy as np

怎么更新呢？
还是通过学习率，进行错误的惩罚

In [8]:
def sigmoid(s):
    return 1/(1+np.exp(-s))

In [9]:
sigmoid(0)

0.5

In [20]:
def cross_entropy_err(X,y,W):
    res=np.dot(X,W)
    return np.dot(sigmoid(-np.dot(y,res.transpose())),-np.dot(y,X))


In [21]:
def GenerateData(mu,d,N):
    '''
    根据传入参数mu，d，N生成 N 个二维向量的数据集s0 和s1
    Args:
        mu:均值
        d(int):每个数据维度
        N(int):每个分类有N条数据
    Returns:
        X(numpy.ndarray):维度为[N*2,d],前N行为一类，后N行为另一类
        y(numpy.ndarray):维度为[N*2]，标注数据的类别
    '''
    I=np.eye(d)
    mu0=np.array([mu,0])
    mu1=np.array([-mu,0])

    s0=np.random.multivariate_normal(mu0,I,N)
    s1=np.random.multivariate_normal(mu1,I,N)
    X=np.zeros([2*N,d])
    X[:N]=s0
    X[N:]=s1
    y=np.zeros(2*N)
    y[:N]=1
    y[N:]=-1
    return X,y

In [28]:
def logistic(learning_rate=0.1):
    X,y=GenerateData(3,2,10)
    d=X.shape[1]
    W=np.zeros(d)
    for i in range(10):
        err=cross_entropy_err(X,y,W)
        if abs(err).max()<1e-6:
            print('achieve optimal')
            print(W)
            break
        W=W-learning_rate*err
        print(err)
        print(W)

In [29]:
logistic()

[-31.8734707    3.81847104]
[ 3.18734707 -0.3818471 ]
achieve optimal
[ 3.18734707 -0.3818471 ]
